# BAA Finish Time Predictor

Based on data used in the Richard Smith et al paper

In [1]:
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
baa1 = pd.read_csv("../TIM.txt",sep=' ') #,nrows=30)

In [8]:
baa1.shape

(69923, 18)

In [9]:
baa1.columns

Index(['BibNum', 'Year', 'Age', 'Gender1F2M', 'StartHr', 'StartMin', 'K0-5',
       'K5-10', 'K10-15', 'K15-20', 'K20-25', 'K25-30', 'K30-35', 'K35-40',
       'K40-Fin', 'HalfMar', 'Unnamed: 16', 'Age2014'],
      dtype='object')

In [10]:
baa1.head()

,BibNum,Year,Age,Gender1F2M,StartHr,StartMin,K0-5,K5-10,K10-15,K15-20,K20-25,K25-30,K30-35,K35-40,K40-Fin,HalfMar,Unnamed: 16,Age2014
0,27040,2013,27,2,9.0,18.08,53.22,55.02,86.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN
1,27041,2013,26,2,9.0,18.07,53.22,54.98,86.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN
2,21551,2013,40,2,10.0,43.12,54.08,50.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,NaN
3,23871,2013,68,2,10.0,45.77,47.03,49.02,53.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,NaN
4,23872,2013,40,2,10.0,45.77,47.03,49.02,53.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,NaN


In [11]:
baa1.groupby(baa1['Year']).count()

,BibNum,Age,Gender1F2M,StartHr,StartMin,K0-5,K5-10,K10-15,K15-20,K20-25,K25-30,K30-35,K35-40,K40-Fin,HalfMar,Unnamed: 16,Age2014
Year,,,,,,,,,,,,,,,,,
2010,22670,22670,22670,22670,22670,22670,22670,22670,22670,22670,22670,22670,22670,22670,22663,0,0
2011,23913,23913,23913,23913,23913,23913,23913,23913,23913,23913,23913,23913,23913,23913,23866,0,0
2013,23340,23340,23340,23322,23322,23322,23309,23283,23226,23131,23096,22633,22102,17584,23205,5754,0


In [12]:
baa1.groupby([baa1['Year'],baa1['Gender1F2M']]).count()

BibNum    Age  StartHr  StartMin   K0-5  K5-10  K10-15  \
Year Gender1F2M                                                           
2010 1             9551   9551     9551      9551   9551   9551    9551   
     2            13119  13119    13119     13119  13119  13119   13119   
2011 1            10074  10074    10074     10074  10074  10074   10074   
     2            13839  13839    13839     13839  13839  13839   13839   
2013 1             9984   9984     9979      9979   9979   9976    9962   
     2            13356  13356    13343     13343  13343  13333   13321   

                 K15-20  K20-25  K25-30  K30-35  K35-40  K40-Fin  HalfMar  \
Year Gender1F2M                                                             
2010 1             9551    9551    9551    9551    9551     9551     9549   
     2            13119   13119   13119   13119   13119    13119    13114   
2011 1            10074   10074   10074   10074   10074    10074    10062   
     2            13839   13839   13839   13839   13839    13839    13804   
2013 1             9933    9881    9866    9618    9312     6943     9919   
     2            13293   13250   13230   13015   12790    10641    13286   

                 Unnamed: 16  Age2014  
Year Gender1F2M                        
2010 1                     0        0  
     2                     0        0  
2011 1                     0        0  
     2                     0        0  
2013 1                  3039        0  
     2                  2715        0

In [13]:
made25 = baa1.loc[baa1['K20-25'].notnull()]

In [16]:
baa1['10_k'] = baa1['K0-5'] + baa1['K5-10']
baa1['15_k'] = baa1['K0-5'] + baa1['K5-10'] + baa1['K10-15']

In [17]:
baa1.head()

,BibNum,Year,Age,Gender1F2M,StartHr,StartMin,K0-5,K5-10,K10-15,K15-20,K20-25,K25-30,K30-35,K35-40,K40-Fin,HalfMar,Unnamed: 16,Age2014,10_k,15_k
0,27040,2013,27,2,9.0,18.08,53.22,55.02,86.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,108.24,194.56
1,27041,2013,26,2,9.0,18.07,53.22,54.98,86.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,108.20,194.55
2,21551,2013,40,2,10.0,43.12,54.08,50.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,NaN,104.08,NaN
3,23871,2013,68,2,10.0,45.77,47.03,49.02,53.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,NaN,96.05,149.62
4,23872,2013,40,2,10.0,45.77,47.03,49.02,53.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,NaN,96.05,149.62


In [19]:
baa1.describe()

,BibNum,Year,Age,Gender1F2M,StartHr,StartMin,K0-5,K5-10,K10-15,K15-20,K20-25,K25-30,K30-35,K35-40,K40-Fin,HalfMar,Unnamed: 16,Age2014,10_k,15_k
count,6.992300e+04,69923.000000,69923.000000,69923.000000,69905.000000,69905.000000,69905.000000,69892.000000,69866.000000,69809.000000,69714.000000,69679.000000,69216.000000,68685.000000,64167.000000,69734.000000,5754.000000,0.0,69891.000000,69865.000000
mean,8.185457e+04,2011.343378,41.622242,1.576548,3.335155,8.201326,25.321808,25.414740,25.550358,26.220238,26.738444,28.286134,29.286019,29.348289,12.509954,108.117269,45.173792,NaN,50.734914,76.279471
std,2.385411e+06,1.241576,10.858319,0.494109,4.713703,15.302713,3.631974,3.847162,4.032132,4.479741,5.004954,5.718730,6.056898,6.109361,2.553475,16.451617,13.104470,NaN,7.412796,11.342154
min,1.000000e+00,2010.000000,18.000000,1.000000,0.000000,0.000000,14.480000,12.410000,12.410000,12.410000,12.410000,12.410000,12.410000,11.440000,2.670000,61.950000,19.000000,NaN,29.100000,43.740000
25%,7.229000e+03,2010.000000,33.000000,1.000000,0.000000,0.000000,22.800000,22.800000,22.870000,23.270000,23.450000,24.500000,25.150000,25.130000,10.880000,96.920000,34.000000,NaN,45.645000,68.520000
50%,1.403400e+04,2011.000000,42.000000,2.000000,0.000000,0.000000,24.970000,24.980000,25.020000,25.550000,25.880000,27.200000,28.130000,28.220000,12.130000,106.100000,45.000000,NaN,49.960000,75.000000
75%,2.080800e+04,2013.000000,49.000000,2.000000,10.000000,4.850000,27.380000,27.380000,27.480000,28.230000,28.870000,30.750000,32.250000,32.480000,13.670000,116.680000,55.000000,NaN,54.780000,82.300000
max,1.000001e+08,2013.000000,83.000000,2.000000,10.000000,59.980000,73.520000,68.220000,86.350000,102.970000,99.930000,112.650000,155.750000,160.950000,202.250000,259.920000,84.000000,NaN,108.240000,194.560000
